# Semicustom digital design demo using OpenROAD

## Sources

### RTL description (Verilog)

The OpenROAD workflow takes the circuit's RTL description as an input. For instance, it can be a three bits XOR gate.


<blockquote><details>

<summary>
    
#### ↕️ Types in Verilog

</summary>
    
```verilog
// Three scalar nets
wire op_b, op_a, result;
// One 16-bit net
wire [15:0] word_bus;
// 1K-array of 8-bit nets
wire [7:0] byte_array [0:1023];
```
    
</details></blockquote>

<blockquote><details>

<summary>
    
#### ↕️ Assignation (non-blocking) in Verilog

</summary>
    
```verilog
// 16-bit, hexadecimal constant
assign address = 16'hCAFE;
// Unsized, decimal constant
assign counter = 'd42;
// 1-bit, binary constant
assign answer = 1'b1;

// Ternary assignation
assign muxed = which ? source_1 : source_2;

// Concatenation
assign padded_packet = {5'b00000,body,suffix};
// Replication
assign odd_mask = {10{2'b10}};

// Indexing
assign one_bit = bus[4];
assign bits = bus[15:12];
```
    
</details></blockquote>

<blockquote><details>

<summary>
    
#### ↕️ Operators in Verilog

</summary>
    
```verilog
// Addition, substraction, negation
assign sum = op_a + op_b; assign sub = op_a + op_b; assign opp = -op_a
// Multiplication, division, modulo
assign prod = op_a * op_b; assign div = op_a / op_b; assign rem = op_a & op_b
    
// Bitwise not, or, and, xor
assign n = ~m; assign a = b | c; assign d = e & f; assign x = y ^ z

// Logical not, and, or
assign ans = !v; assign ans = v || w; assign ans = v && w;
// Logical equality, difference
assign ans = v == w; assign ans = v != w;
// Relations (strictly) greater, (strictly) lower than
assign sg = a > b; assign gt = a >= b; assign sl = a < b; assign lt = a <= b;
    
// Left, right shift by n bits
assign l << n; assign r >> n;
// Left, right arithmetic shift by n bits
assign l <<< n; assign r >>> n;
```
    
</details></blockquote>

In [ ]:
%%writefile v/xor3.v
module xor3(
    input wire a,
    input wire b,
    input wire c,
    output wire out
);
    assign out = a ^ b ^ c;
endmodule

### Configuration file (JSON)

A configuration file should be provided. It describes constraints and strategies applied during synthesis and implementation of the circuit.

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "xor3",
    "VERILOG_FILES": "dir::v/xor3.v",
    "CLOCK_TREE_SYNTH": false,
    "CLOCK_PORT": null,
    "PL_RANDOM_GLB_PLACEMENT": true,
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 40 40",
    "PL_TARGET_DENSITY": 0.8,
    "FP_PDN_AUTO_ADJUST": false,
    "FP_PDN_VPITCH": 10,
    "FP_PDN_HPITCH": 10,
    "FP_PDN_VOFFSET": 5,
    "FP_PDN_HOFFSET": 5,
    "DIODE_INSERTION_STRATEGY": 3
}

## Workflow
The provided `flow.tcl` is a script describing the OpenROAD workflow. A _GDS_ file will be generated using the RTL circuit description, the PDK and the configuration file.

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

## Output products

### Display layout

The implemented layout can be retrieved as follows:

In [ ]:
import glob
import gdstk
import IPython.display

gdsii = sorted(glob.glob("./runs/*/results/final/gds/*.gds"))[-1]
top = gdstk.read_gds(gdsii).top_level()
top[0].write_svg('svg/inverter.svg')
IPython.display.SVG('svg/inverter.svg')

### Reporting

Many reports are available under:

```
freeechips/semicustom/runs/RUN_YYYY.MM.DD_HH.MM.SS/reports/.
```

An overview of the main figures can be retrieved as well:

In [ ]:
import glob
import pandas as pd

metrics = pd.read_csv(sorted(glob.glob("./runs/*/reports/metrics.csv"))[-1])

print(f"tritonRoute_violations  {metrics['tritonRoute_violations'][0]}")
print(f"Short_violations        {metrics['Short_violations'][0]}")
print(f"OffGrid_violations      {metrics['OffGrid_violations'][0]}")
print(f"MinHole_violations      {metrics['MinHole_violations'][0]}")
print(f"Other_violations        {metrics['Other_violations'][0]}")
print(f"Magic_violations        {metrics['Magic_violations'][0]}")
print(f"pin_antenna_violations  {metrics['pin_antenna_violations'][0]}")
print(f"net_antenna_violations  {metrics['net_antenna_violations'][0]}")
print(f"lvs_total_errors        {metrics['lvs_total_errors'][0]}")
print(f"cvc_total_errors        {metrics['cvc_total_errors'][0]}")
print(f"klayout_violations      {metrics['klayout_violations'][0]}")

# To have fun going further...

## Sequential circuits in Verilog

The tutorial above focuses on combinational circuits. Sequential circuits can obviously be described in Verilog as well. Sequential blocks feature an `always` block. Refer to the vendor's reference to get the sensitivity list's syntax (e.g. to determine if reset is synchronous or asynchronous).

<blockquote><details>

<summary>
    
#### ↕️ Sequential structures in Verilog

</summary>

```verilog
always @(posedge clk or negedge rst) begin
  if (rst) begin 
    counter <= 0;
  end else begin
    counter <= counter + 1;
  end
end
```
    
</details></blockquote>

Advanced structures like the `case` structure can be used to describe finite state machines. FSM decoders can be described in an abstract way using `always` blocks in a fully combinational way:

<blockquote><details>

<summary>
    
#### ↕️ Advanced structures in Verilog

</summary>

```verilog
    
/*
 *   Case structure
 */
    
case (state)

  3'b000: idle_led <= 1'b1;
  
  3'b001,
  3'b010: work_led <= 1'b1;
  
  3'b011: begin
    muxed <= spi_1;
    work_led <= 1'b1;
  end
  
  default: begin
    muxed <= 0;
    work_led <= 1'b0; 
    idle_led <= 1'b1;
  end

endcase

/*
 *   Combinational always structure
 *     To describe priorities in a procedural fashion,
 *     use blocking `<=` assignations instead of
 *     non-blocking `=` assignations.
 */
                
always @( * ) begin
  flag <= 1'b0;
  if (error) begin 
    flag <= 1'b1;
  end
end
```

</details></blockquote>

## High-Level Synthesis (HLS)

RTL description of circuits does not follow an imperative programming paradigm. It is a description language that produces highly parallelized designs.

High-Level Synthesis provides an imperative language and a compiler that synthesizes the imperative instructions into RTL. For instance, _XLS_ provides a _Rust_-like language:

In [ ]:
%%writefile xls/x/adder.x
pub fn adder(in1: u1, in2: u1) -> u2 {
  let sum: u2 = in1 as u2 + in2 as u2;
  sum
}

#[test]
fn adder_test() {
  let _= assert_eq(adder(u1:0, u1:0), u2:0b00);
  let _= assert_eq(adder(u1:0, u1:1), u2:0b01);
  let _= assert_eq(adder(u1:1, u1:1), u2:0b10);
  _
}

Testing, parsing and linting can be performed prior to RTL synhesis:

In [ ]:
!interpreter_main xls/x/adder.x

Now that the imperative instructions are tested, the RTL design can be synthesized by _XLS_:

In [ ]:
XLS_DESIGN_NAME = 'adder'
!ir_converter_main --top={XLS_DESIGN_NAME} xls/x/{XLS_DESIGN_NAME}.x > xls/ir/{XLS_DESIGN_NAME}.ir
!opt_main xls/ir/{XLS_DESIGN_NAME}.ir > xls/ir/{XLS_DESIGN_NAME}_opt.ir
!codegen_main --generator=combinational xls/ir/{XLS_DESIGN_NAME}_opt.ir > v/{XLS_DESIGN_NAME}.v
!cat v/{XLS_DESIGN_NAME}.v

# References
Inspired from:
“Silicon Notebooks.” CHIPS Alliance, Apr. 08, 2023. Accessed: Apr. 10, 2023. [Online]. Available: https://github.com/chipsalliance/silicon-notebooks/blob/b65134a43b01ae31423f7ee87110740b2257ac42/digital-inverter-openlane.ipynb (Apache License 2.0)